<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-Basic-Configurations" data-toc-modified-id="Imports-and-Basic-Configurations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and Basic Configurations</a></span><ul class="toc-item"><li><span><a href="#For-Google-COLAB" data-toc-modified-id="For-Google-COLAB-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>For Google COLAB</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Imports</a></span></li></ul></li><li><span><a href="#Used-Classes-and-Functions" data-toc-modified-id="Used-Classes-and-Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Used Classes and Functions</a></span><ul class="toc-item"><li><span><a href="#Helper-Functions" data-toc-modified-id="Helper-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Helper Functions</a></span><ul class="toc-item"><li><span><a href="#LoadWorkState-+-StoreWorkState" data-toc-modified-id="LoadWorkState-+-StoreWorkState-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>LoadWorkState + StoreWorkState</a></span></li><li><span><a href="#LogEvent-+-LogPaths-+-LogBestSolution" data-toc-modified-id="LogEvent-+-LogPaths-+-LogBestSolution-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>LogEvent + LogPaths + LogBestSolution</a></span></li><li><span><a href="#GetCurrentState" data-toc-modified-id="GetCurrentState-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>GetCurrentState</a></span></li><li><span><a href="#LoadCreateSplitFile" data-toc-modified-id="LoadCreateSplitFile-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>LoadCreateSplitFile</a></span></li><li><span><a href="#HandleCurrentState" data-toc-modified-id="HandleCurrentState-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>HandleCurrentState</a></span></li><li><span><a href="#PlotFigure-+-PlotRandomFigure-+-PlotHistory" data-toc-modified-id="PlotFigure-+-PlotRandomFigure-+-PlotHistory-2.1.6"><span class="toc-item-num">2.1.6&nbsp;&nbsp;</span>PlotFigure + PlotRandomFigure + PlotHistory</a></span></li><li><span><a href="#External-Metrics" data-toc-modified-id="External-Metrics-2.1.7"><span class="toc-item-num">2.1.7&nbsp;&nbsp;</span>External Metrics</a></span></li><li><span><a href="#Levy" data-toc-modified-id="Levy-2.1.8"><span class="toc-item-num">2.1.8&nbsp;&nbsp;</span>Levy</a></span></li><li><span><a href="#MapSolutionAsString-+-MapSolutionForCNN" data-toc-modified-id="MapSolutionAsString-+-MapSolutionForCNN-2.1.9"><span class="toc-item-num">2.1.9&nbsp;&nbsp;</span>MapSolutionAsString + MapSolutionForCNN</a></span></li><li><span><a href="#BuildCompileTransferLearningCNN" data-toc-modified-id="BuildCompileTransferLearningCNN-2.1.10"><span class="toc-item-num">2.1.10&nbsp;&nbsp;</span>BuildCompileTransferLearningCNN</a></span></li></ul></li><li><span><a href="#Fitness-Functions" data-toc-modified-id="Fitness-Functions-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Fitness Functions</a></span></li><li><span><a href="#PopulationBasedOptimizer-Class" data-toc-modified-id="PopulationBasedOptimizer-Class-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>PopulationBasedOptimizer Class</a></span></li><li><span><a href="#MantaRayForagingOptimizer-Class" data-toc-modified-id="MantaRayForagingOptimizer-Class-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>MantaRayForagingOptimizer Class</a></span></li></ul></li><li><span><a href="#Learning-and-Optimization-Configurations" data-toc-modified-id="Learning-and-Optimization-Configurations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Learning and Optimization Configurations</a></span></li><li><span><a href="#PBO-Checking-and-Object-Creation" data-toc-modified-id="PBO-Checking-and-Object-Creation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>PBO Checking and Object Creation</a></span></li><li><span><a href="#Run" data-toc-modified-id="Run-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Run</a></span></li></ul></div>

# Imports and Basic Configurations

## For Google COLAB

In [ ]:
import os, random

TF_VERSION = "latest"
NUM_THREADS = 1
DEBUG = True
COLAB = False
RANDOM_SEED = random.randint(1, 1e4) # 1337
BASE_DIR = os.getcwd()
COLAB_DIR = ""

if COLAB:
  BASE_DIR = COLAB_DIR
  from google.colab import drive
  drive.mount("/content/drive")
  if (TF_VERSION == "latest"):
    %tensorflow_version 2.x

## Imports

In [ ]:
from abc import ABCMeta, abstractmethod
import numpy as np
import matplotlib.pyplot as plt
import cv2, os, itertools, random, pickle, warnings, datetime, math, gc, copy
import timeit, traceback, contextlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, applications, callbacks, metrics
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU, PReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import L1L2, l2, l1, l1_l2
from tensorflow.keras.models import load_model
from keras.utils.vis_utils import plot_model

from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, \
  EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, \
  EfficientNetB6, EfficientNetB7, MobileNet, MobileNetV2, \
  VGG16, VGG19, ResNet50, ResNet101, ResNet152, \
  ResNet50V2, ResNet101V2, ResNet152V2, Xception, DenseNet121, DenseNet169, \
  DenseNet201, NASNetMobile, NASNetLarge, InceptionV3, InceptionResNetV2, \
  MobileNetV3Small, MobileNetV3Large
from tensorflow.keras.activations import elu, relu, selu, gelu, sigmoid, \
  hard_sigmoid, linear, softmax, softplus, softsign, swish, tanh, \
  exponential
from tensorflow.keras.optimizers import Adadelta, Nadam, Adam, \
  Adamax, Adagrad, Ftrl, SGD, RMSprop
from tensorflow.keras.initializers import GlorotNormal, GlorotUniform, Zeros, \
  HeNormal, HeUniform, LecunNormal, LecunUniform, Identity, \
  Ones, Orthogonal, TruncatedNormal, VarianceScaling, \
  RandomNormal, RandomUniform
from tensorflow.keras.metrics import Accuracy, AUC, Precision, \
  Recall, CosineSimilarity, CategoricalCrossentropy, KLDivergence, \
  CategoricalHinge, Hinge, SquaredHinge, Poisson, TruePositives, \
  TrueNegatives, FalsePositives, FalseNegatives, BinaryAccuracy, \
  BinaryCrossentropy, CategoricalAccuracy, LogCoshError, Mean, \
  MeanAbsoluteError, MeanAbsolutePercentageError, MeanIoU, \
  MeanSquaredError, MeanSquaredLogarithmicError, \
  MeanTensor, RootMeanSquaredError, SparseCategoricalAccuracy, \
  SparseCategoricalCrossentropy, SparseTopKCategoricalAccuracy, Sum, \
  TopKCategoricalAccuracy

# Used Classes and Functions

## Helper Functions

### LoadWorkState + StoreWorkState

In [ ]:
def LoadWorkState(statePickle):
  f = open(statePickle, "rb")
  L = pickle.load(f)
  f.close()
  return L


def StoreWorkState(L, statePickle):
  f = open(statePickle, "wb")
  L = pickle.dump(L, f)
  f.close()

### LogEvent + LogPaths + LogBestSolution

In [ ]:
def LogEvent(loggerFile, data, storeTimestamp=True):
  f = open(loggerFile, "a")
  if (storeTimestamp == True):
    now = str(datetime.datetime.now())
    f.write(f"Timestamp {now}:\t")
  f.write(f"{str(data)}\n")
  f.close()


def LogPaths(currentState, iteration, points, score, storeTimestamp=True):
  if (not os.path.exists(currentState["PATHS_FILE"])):
    f = open(currentState["PATHS_FILE"], "a")
    f.write("Iteration,Points,Score,Timestamp")
    f.write(f"\n")
    f.close()
    
  f = open(currentState["PATHS_FILE"], "a")
  f.write(f"{iteration},")
  strPoints = "_".join([str(p) for p in list(points)])
  f.write(f"{strPoints},")
  f.write(f"{score},")
  if (storeTimestamp == True):
    now = str(datetime.datetime.now())
    f.write(f"{now}")
  f.write(f"\n")
  f.close()
  
  
def LogBestSolution(currentState, iteration, L, storeTimestamp=True):
  if (not os.path.exists(currentState["BEST_FILE"])):
    f = open(currentState["BEST_FILE"], "a")
    f.write("#,Params Optimizer,Batch Size,Dropout,Model Learn Ratio,")
    f.write("Hidden Activation,Params Initializer,Regularizer,Loss,")
    for i in range(6):
      f.write(currentState['METRICS_KEYWORDS'][i] + ",")
    f.write("WS,")
    for i in range(6, len(currentState['METRICS_KEYWORDS'])):
      f.write(currentState['METRICS_KEYWORDS'][i] + ",")
    f.write("Timestamp")
    f.write(f"\n")
    f.close()

  f = open(currentState["BEST_FILE"], "a")
  f.write(f"{str(iteration)},")
  for el in L:
    f.write(f"{str(el)},")
  if (storeTimestamp == True):
    now = str(datetime.datetime.now())
    f.write(f"{now}")
  f.write(f"\n")
  f.close()

### GetCurrentState

In [ ]:
def GetCurrentState():
  L = {
    "TF_VERSION": TF_VERSION,
    "RANDOM_SEED": RANDOM_SEED,
    "DEBUG": DEBUG,
    "COLAB": COLAB,
    "BASE_DIR": BASE_DIR,
    "COLAB_DIR": COLAB_DIR,
    "OWNER": OWNER,
    "PBO": PBO,
    "MODEL": MODEL,
    "DATASET_PICKLE_FILE": DATASET_PICKLE_FILE,
    "PICKELS_BASE_DIR": PICKELS_BASE_DIR,
    "RESULTS_BASE_DIR": RESULTS_BASE_DIR,
    "IMAGE_SHAPE": IMAGE_SHAPE,
    "CATEGORIES": CATEGORIES,
    "ITERS": ITERS,
    "EPOCHS": EPOCHS,
    "POPULATION_SIZE": POPULATION_SIZE,
    "DIM": DIM,
    "SKIP": SKIP,
    "POPULATION": POPULATION,
    "IS_NEW": IS_NEW,
    "TRAIN_RATIO": TRAIN_RATIO,
    "DROPOUT_RANGE": DROPOUT_RANGE,
    "OUTPUT_ACTIVATION": OUTPUT_ACTIVATION,
    "PRETRAINED_WEIGHTS": PRETRAINED_WEIGHTS,
    "BATCH_SIZES": BATCH_SIZES,
    "LEARN_RATIOS": LEARN_RATIOS,
    "OPTIMIZERS": OPTIMIZERS,
    "HIDDEN_ACTIVATIONS": HIDDEN_ACTIVATIONS,
    "WEIGHT_INITIALIZERS": WEIGHT_INITIALIZERS,
    "RESULTS_DIR": RESULTS_DIR,
    "DATASET_PICKLE": DATASET_PICKLE,
    "OWNER_DIR": OWNER_DIR,
    "HDF5_RESULTS_DIR": HDF5_RESULTS_DIR,
    "FIGURES_RESULTS_DIR": FIGURES_RESULTS_DIR,
    "HISTORY_RESULTS_DIR": HISTORY_RESULTS_DIR,
    "STATE_PICKLE": STATE_PICKLE,
    "LOGGER_FILE": LOGGER_FILE,
    "BEST_FILE": BEST_FILE,
    "PATHS_FILE": PATHS_FILE,
    "SPLIT_DATASET_PICKLE": SPLIT_DATASET_PICKLE,
    "AUGMENTATION_CONFIG": AUGMENTATION_CONFIG,
    "GA": {},
    "MRFO": {},
    "HHO": {
      "RABBIT_LOCATION": None,
      "CONVERGENCE_CURVE": None,
      "RABBIT_ENERGY": float("-inf")
    },
    "INTERPOLATION": INTERPOLATION,
    "USE_TF": USE_TF,
    "REGULARIZERS": REGULARIZERS,
    "LOSS": LOSS,
    "METRICS_KEYWORDS": METRICS_KEYWORDS,
    "METRICS_NAMES": METRICS_NAMES,
    "IS_DL": IS_DL,
    "IS_MAX_PROBLEM": IS_MAX_PROBLEM,
    "LOWER_BOUND": LOWER_BOUND,
    "UPPER_BOUND": UPPER_BOUND,
  }
  return L

### LoadCreateSplitFile

In [ ]:
def LoadCreateSplitFile(datasetPickle,
            splitPickle,
            noOfClasses,
            trainRatio,
            debug=True):
  if (os.path.exists(splitPickle)):
    if (debug): print(f"Loading {splitPickle}")
    f = open(splitPickle, "rb")
    [Xtrain, Xvalidation, Xtest, Ytrain, Yvalidation,
     Ytest] = pickle.load(f)
    f.close()
    Q = [Xtrain, Xvalidation, Xtest, Ytrain, Yvalidation, Ytest]
  else:
    if (debug): print(f"Creating {splitPickle}")
    randomState = 42
    f = open(datasetPickle, "rb")
    dataset = pickle.load(f)
    f.close()
    random.shuffle(dataset)
    images, classes = zip(*dataset)
    images = np.array(images, dtype='float32')
    images = (images / 255.0).astype('float32')
    classes = to_categorical(classes, noOfClasses)
    classes = np.array(classes)
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(
      images, classes, train_size=trainRatio, random_state=randomState)
    Xtrain, Xvalidation, Ytrain, Yvalidation = train_test_split(
      Xtrain, Ytrain, train_size=trainRatio, random_state=randomState)
    Q = [Xtrain, Xvalidation, Xtest, Ytrain, Yvalidation, Ytest]
    f = open(splitPickle, "wb")
    pickle.dump(Q, f)
    f.close()
  return Q

### HandleCurrentState

In [ ]:
def HandleCurrentState(isNew, statePickle, loggerFile, debug=True):
  if (isNew):
    currentState = GetCurrentState()
    StoreWorkState(currentState, statePickle)
    state = "The current state is created as required!"
    LogEvent(loggerFile, state)
    LogEvent(loggerFile, currentState)
  else:
    if (os.path.exists(statePickle)):
      currentState = LoadWorkState(statePickle)
      state = "The current state is loaded as the state file is found!"
      LogEvent(loggerFile, state)
      LogEvent(loggerFile, currentState)
    else:
      currentState = GetCurrentState()
      StoreWorkState(currentState, statePickle)
      state = "The current state is created again as the state file is not found!"
      LogEvent(loggerFile, state)
      LogEvent(loggerFile, currentState)
  if (debug): print(state)
  if (debug): print(str(currentState))
  return currentState

### PlotFigure + PlotRandomFigure + PlotHistory

In [ ]:
def PlotFigure(image, category, debug=True, dpi=100):
  fig = plt.figure(dpi=dpi)
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  if (category): plt.title(f"Image Label '{category}'")
  plt.xticks([])
  plt.yticks([])
  plt.show()
  plt.clf()
  plt.close()


def PlotRandomFigure(X, y, categories, debug=True):
  rand = np.random.randint(0, X.shape[0])
  image = X[rand]
  category = categories[np.argmax(y[rand])]
  if (debug):
    print("Max:", np.max(image))
    print("Min:", np.min(image))
    print("Mean:", np.mean(image))
    print("Standard Deviation:", np.std(image))
  PlotFigure(image, category, debug)


def PlotHistory(currentState, figPath, history, figSize=(25, 15)):
  fig = plt.figure(num=None, figsize=figSize)
  keywords = copy.copy(currentState["METRICS_KEYWORDS"])
  keywords.insert(0, "Loss")
  figs = copy.copy(currentState["METRICS_NAMES"])
  figs.insert(0, "loss")
  locs = ["lower"] * len(keywords)
  locs[0] = "upper"
  titles = ("(a)", "(b)", "(c)", "(d)", "(e)", "(f)", "(g)", "(h)", "(i)",
        "(j)", "(k)", "(l)", "(m)", "(n)", "(o)", "(p)", "(q)", "(r)",
        "(s)", "(t)", "(v)", "(w)", "(x)", "(y)", "(z)")
  for i in range(7):  # The loss and first 6 metrics.
    keyword = keywords[i]
    plt.subplot(2, 4, i + 1)  # Don't forget to handle 2 x 4.
    plt.plot(history.history[figs[i]], label=keyword)
    plt.plot(history.history[f'val_{figs[i]}'],
         label=f'Validation {keyword}')
    plt.xlabel('Epoch')
    plt.ylabel(keyword)
    plt.grid("both")
    plt.title(f"{titles[i]} {keywords[i]} Curves")
    plt.legend(loc=f'{locs[i]} right')
  plt.tight_layout()
  plt.savefig(figPath, dpi=500)
  #plt.show()
  plt.clf()
  plt.close()

### External Metrics

In [ ]:
def iou_coef_m(yTrue, yPred, smooth=1):
  intersection = K.sum(K.abs(yTrue * yPred), axis=-1)
  union = K.sum(yTrue, axis=-1) + K.sum(yPred, axis=-1) - intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou


def dice_coef_m(yTrue, yPred, smooth=1):
  intersection = K.sum(yTrue * yPred, axis=-1)
  union = K.sum(yTrue, axis=-1) + K.sum(yPred, axis=-1)
  dice = K.mean((2.0 * intersection + smooth) / (union + smooth), axis=0)
  return dice


def sensitivity_m(yTrue, yPred):
  truePositives = K.sum(K.round(K.clip(yTrue * yPred, 0, 1)))
  possiblePositives = K.sum(K.round(K.clip(yTrue, 0, 1)))
  sensitivity = truePositives / (possiblePositives + K.epsilon())
  return sensitivity


def specificity_m(yTrue, yPred):
  trueNegatives = K.sum(K.round(K.clip((1.0 - yTrue) * (1.0 - yPred), 0, 1)))
  possibleNegatives = K.sum(K.round(K.clip(1 - yTrue, 0, 1)))
  specificity = trueNegatives / (possibleNegatives + K.epsilon())
  return specificity


def recall_m(yTrue, yPred):
  truePositives = K.sum(K.round(K.clip(yTrue * yPred, 0, 1)))
  possiblePositives = K.sum(K.round(K.clip(yTrue, 0, 1)))
  recall = truePositives / (possiblePositives + K.epsilon())
  return recall


def precision_m(yTrue, yPred):
  truePositives = K.sum(K.round(K.clip(yTrue * yPred, 0, 1)))
  predictedPositives = K.sum(K.round(K.clip(yPred, 0, 1)))
  precision = truePositives / (predictedPositives + K.epsilon())
  return precision


def f1_m(yTrue, yPred):
  precision = precision_m(yTrue, yPred)
  recall = recall_m(yTrue, yPred)
  return 2.0 * ((precision * recall) / (precision + recall + K.epsilon()))

### Levy

In [ ]:
def Levy(dim):
  beta = 1.5
  gamma = math.gamma(1.0 + beta)
  num = gamma * math.sin(math.pi * beta / 2.0)
  den = (gamma / 2.0) * beta * (2.0**((beta - 1.0) / 2.0))
  sigma = (num / den)**(1.0 / beta)
  u = 0.01 * np.random.randn(dim) * sigma
  v = np.random.randn(dim)
  zz = np.power(np.absolute(v), (1.0 / beta))
  step = np.divide(u, zz)
  return step

### MapSolutionAsString + MapSolutionForCNN

In [ ]:
def MapSolutionAsString(currentState, solution):
  if (not currentState["IS_DL"]):
    return list(copy.copy(solution))
  
  mappedSolution = None
  if (isinstance(currentState["DIM"], int)):
    mappedSolution = [None, None, None, None, None, None, None]
    for i in range(currentState["DIM"]):
      mappedSolution[i] = "auto"
  elif (isinstance(currentState["DIM"], list)
      or isinstance(currentState["DIM"], tuple)):
    mappedSolution = copy.copy(currentState["DIM"])
    for i in range(len(mappedSolution)):
      if (mappedSolution[i] == "N/A"): mappedSolution[i] = None
  if (mappedSolution[0] == "auto"):
    r1 = math.floor(solution[0] * (len(currentState["OPTIMIZERS"]) - 1))
    optimizer = currentState["OPTIMIZERS"][r1]
    mappedSolution[0] = optimizer
  if (mappedSolution[1] == "auto"):
    r2 = math.floor(solution[1] * (len(currentState["BATCH_SIZES"]) - 1))
    batchSize = currentState["BATCH_SIZES"][r2]
    mappedSolution[1] = batchSize
  if (mappedSolution[2] == "auto"):
    low = currentState["DROPOUT_RANGE"][0]
    high = currentState["DROPOUT_RANGE"][1]
    step = 0.25
    dropouts = np.arange(low, high + step, step) / 10.0
    rr = math.floor(solution[2] * (len(dropouts) - 1))
    dropout = dropouts[rr]
    mappedSolution[2] = dropout
  if (mappedSolution[3] == "auto"):
    r3 = math.floor(solution[3] * (len(currentState["LEARN_RATIOS"]) - 1))
    learnRatio = currentState["LEARN_RATIOS"][r3]
    mappedSolution[3] = learnRatio
  if (mappedSolution[4] == "auto"):
    r4 = math.floor(solution[4] *
            (len(currentState["HIDDEN_ACTIVATIONS"]) - 1))
    activation = currentState["HIDDEN_ACTIVATIONS"][r4]
    mappedSolution[4] = activation
  if (mappedSolution[5] == "auto"):
    r5 = math.floor(solution[5] *
            (len(currentState["WEIGHT_INITIALIZERS"]) - 1))
    weightInitializer = currentState["WEIGHT_INITIALIZERS"][r5]
    mappedSolution[5] = weightInitializer
  if (mappedSolution[6] == "auto"):
    r6 = math.floor(solution[6] * (len(currentState["REGULARIZERS"]) - 1))
    regularizer = currentState["REGULARIZERS"][r6]
    mappedSolution[6] = regularizer
  return copy.copy(mappedSolution)


def MapSolutionForCNN(currentState, solution):
  mappedSolution = MapSolutionAsString(currentState, solution)

  if (mappedSolution[4] == "prelu"):
    mappedSolution[4] = PReLU()
  elif (mappedSolution[4] == "leakyrelu"):
    mappedSolution[4] = LeakyReLU()

  if (mappedSolution[6] is not None):
    splitted = mappedSolution[6].split("_")
    if (splitted[0] == "l1"): regularizer = l1(float(splitted[1]))
    elif (splitted[0] == "l2"): regularizer = l2(float(splitted[1]))
    else: regularizer = None
    mappedSolution[6] = regularizer

  return copy.copy(mappedSolution)

### BuildCompileTransferLearningCNN

In [ ]:
def BuildCompileTransferLearningCNN(model,
                  inputShape,
                  noOfClasses,
                  hiddenActivation,
                  outputActivation,
                  weightInitializer,
                  pretrainedWeights,
                  optimizer,
                  dropout,
                  learnRatio,
                  regularizer,
                  loss,
                  debug=True):

  baseModel = None
  includeTop = False

  if (model == "MobileNetV2"):
    baseModel = MobileNetV2(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "MobileNet"):
    baseModel = MobileNet(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "MobileNetV3Small"):
    baseModel = MobileNetV3Small(weights=pretrainedWeights,
                   include_top=includeTop)
  elif (model == "MobileNetV3Large"):
    baseModel = MobileNetV3Large(weights=pretrainedWeights,
                   include_top=includeTop)
  elif (model == "VGG16"):
    baseModel = VGG16(weights=pretrainedWeights, include_top=includeTop)
  elif (model == "VGG19"):
    baseModel = VGG19(weights=pretrainedWeights, include_top=includeTop)
  elif (model == "ResNet50"):
    baseModel = ResNet50(weights=pretrainedWeights, include_top=includeTop)
  elif (model == "ResNet101"):
    baseModel = ResNet101(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "ResNet152"):
    baseModel = ResNet152(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "ResNet50V2"):
    baseModel = ResNet50V2(weights=pretrainedWeights,
                 include_top=includeTop)
  elif (model == "ResNet101V2"):
    baseModel = ResNet101V2(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "ResNet152V2"):
    baseModel = ResNet152V2(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "Xception"):
    baseModel = Xception(weights=pretrainedWeights, include_top=includeTop)
  elif (model == "DenseNet121"):
    baseModel = DenseNet121(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "DenseNet169"):
    baseModel = DenseNet169(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "DenseNet201"):
    baseModel = DenseNet201(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "NASNetLarge"):
    baseModel = NASNetLarge(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "InceptionV3"):
    baseModel = InceptionV3(weights=pretrainedWeights,
                include_top=includeTop)
  elif (model == "InceptionResNetV2"):
    baseModel = InceptionResNetV2(weights=pretrainedWeights,
                    include_top=includeTop)
  elif (model == "NASNetMobile"):
    baseModel = NASNetMobile(weights=pretrainedWeights,
                 include_top=includeTop)
  elif (model == "EfficientNetB0"):
    baseModel = EfficientNetB0(weights=pretrainedWeights,
                   include_top=includeTop)
  elif (model == "EfficientNetB1"):
    baseModel = EfficientNetB1(weights=pretrainedWeights,
                   include_top=includeTop)
  elif (model == "EfficientNetB2"):
    baseModel = EfficientNetB2(weights=pretrainedWeights,
                   include_top=includeTop)
  elif (model == "EfficientNetB3"):
    baseModel = EfficientNetB3(weights=pretrainedWeights,
                   include_top=includeTop)
  elif (model == "EfficientNetB4"):
    baseModel = EfficientNetB4(weights=pretrainedWeights,
                   include_top=includeTop)
  elif (model == "EfficientNetB5"):
    baseModel = EfficientNetB5(weights=pretrainedWeights,
                   include_top=includeTop)
  elif (model == "EfficientNetB6"):
    baseModel = EfficientNetB6(weights=pretrainedWeights,
                   include_top=includeTop)
  elif (model == "EfficientNetB7"):
    baseModel = EfficientNetB7(weights=pretrainedWeights,
                   include_top=includeTop)

  assert (baseModel is not None)

  if (learnRatio is None): learnRatio = 100.0

  for layer in baseModel.layers:
    layer.trainable = False
  L = len(baseModel.layers)
  fromL = int(learnRatio * L / 100.0)
  for layer in baseModel.layers[L - fromL:]:
    layer.trainable = True

  if (dropout is None or dropout <= 0):
    model = tf.keras.Sequential([
      baseModel,
      layers.GlobalAveragePooling2D(),
      layers.Dense(noOfClasses, activation=outputActivation)
    ])
  else:
    model = tf.keras.Sequential([
      baseModel,
      layers.GlobalAveragePooling2D(),
      layers.Dropout(dropout),
      layers.Dense(noOfClasses, activation=outputActivation)
    ])

  if (optimizer is not None):
    model.compile(optimizer=optimizer, loss=loss, metrics=METRICS)
  else:
    model.compile(loss=loss, metrics=METRICS)
  return model

## Fitness Functions

In [ ]:
def FitnessFunction(iteration, solution, currentState):
  if (currentState["DEBUG"]): print("=" * 50)
  ResetEnvironment()

  mappedSolution = MapSolutionForCNN(currentState, solution)
  optimizer, batchSize, dropout, learnRatio, hiddenActivation, weightInitializer, regularizer = mappedSolution
  mappedSolutionStr = MapSolutionAsString(currentState, solution)
  optimizerStr, batchSizeStr, dropoutStr, learnRatioStr, hiddenActivationStr, weightInitializerStr, regularizerStr = mappedSolutionStr

  aug = ImageDataGenerator(
    rotation_range=currentState["AUGMENTATION_CONFIG"]['rotation_range'],
    width_shift_range=currentState["AUGMENTATION_CONFIG"]
    ['width_shift_range'],
    height_shift_range=currentState["AUGMENTATION_CONFIG"]
    ['height_shift_range'],
    shear_range=currentState["AUGMENTATION_CONFIG"]['shear_range'],
    zoom_range=currentState["AUGMENTATION_CONFIG"]['zoom_range'],
    horizontal_flip=currentState["AUGMENTATION_CONFIG"]['horizontal_flip'],
    vertical_flip=currentState["AUGMENTATION_CONFIG"]['vertical_flip'],
    brightness_range=currentState["AUGMENTATION_CONFIG"]
    ['brightness_range'],
    fill_mode=currentState["AUGMENTATION_CONFIG"]['fill_mode'],
  )

  trainIter = aug.flow(Xtrain, Ytrain, batch_size=batchSize)
  owner = currentState["OWNER"]
  modelName = currentState["MODEL"]
  inputShape = currentState["IMAGE_SHAPE"]
  noOfClasses = len(currentState["CATEGORIES"])
  loss = currentState["LOSS"]

  if (optimizerStr is None): optimizerStr = "NA"
  if (batchSizeStr is None): batchSizeStr = "NA"
  if (dropoutStr is None): dropoutStr = "NA"
  if (learnRatioStr is None): learnRatioStr = "NA"
  if (hiddenActivationStr is None): hiddenActivationStr = "NA"
  if (weightInitializerStr is None): weightInitializerStr = "NA"
  if (regularizerStr is None): regularizerStr = "NA"

  temp1 = f"{str(owner)}_{str(modelName)}_{str(inputShape)}_{str(noOfClasses)}"
  temp1 += f"_{optimizerStr}_{str(batchSizeStr)}_{str(dropoutStr)}_{str(learnRatioStr)}"
  temp1 += f"_{str(hiddenActivationStr)}_{str(weightInitializerStr)}_{str(regularizerStr)}"
  temp2 = temp1 + f"_{str(iteration)}"

  name1 = f"{temp1}.hdf5"
  name2 = f"{temp2}.hdf5"
  figName = f"{temp2}.jpg"
  csvName = f"{temp1}.csv"
  modelPlot = f"model_plot.png"

  checkpointFilepath1 = os.path.join(currentState["HDF5_RESULTS_DIR"], name1)
  checkpointFilepath2 = os.path.join(currentState["HDF5_RESULTS_DIR"], name2)
  figPath = os.path.join(currentState["FIGURES_RESULTS_DIR"], figName)
  csvPath = os.path.join(currentState["HISTORY_RESULTS_DIR"], csvName)
  modelPlotPath = os.path.join(currentState["OWNER_DIR"], modelPlot)

  modelCheckpointCallback1 = callbacks.ModelCheckpoint(
    filepath=checkpointFilepath1,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=0)
  csvLoggerCallback = callbacks.CSVLogger(csvPath, append=True)
  earlyStoppingCallback = callbacks.EarlyStopping(monitor='loss', patience=5)
  callbacksList = [
    modelCheckpointCallback1, csvLoggerCallback, earlyStoppingCallback
  ]

  toLog = f"New {temp1}."
  if (os.path.exists(checkpointFilepath1)):
    toLog = f"Updating using {temp1}."
    if (currentState["DEBUG"]): print(toLog)
    model = load_model(checkpointFilepath1,
               compile=False)  
    if (optimizer is not None):
      model.compile(optimizer=optimizer, loss=LOSS, metrics=METRICS)
    else:
      model.compile(loss=LOSS, metrics=metricsList)
    model.load_weights(checkpointFilepath1)
    ev = model.evaluate(Xvalidation, Yvalidation, verbose=0)
    modelCheckpointCallback1.best = ev[1]  # Accuracy.
    message = f"Start the Model Checkpoint with a Best Value of {ev[1]}."
    if (currentState["DEBUG"]):
      print(message)
    LogEvent(currentState["LOGGER_FILE"], message)
  else:
    ev = modelCheckpointCallback1.best
    message = f"Start the Model Checkpoint with a Best Value of {ev}."
    if (currentState["DEBUG"]):
      print(toLog)
      print(message)
    LogEvent(currentState["LOGGER_FILE"], message)
    if (currentState["USE_TF"]):
      model = BuildCompileTransferLearningCNN(
        modelName,
        inputShape,
        noOfClasses,
        hiddenActivation,
        currentState["OUTPUT_ACTIVATION"],
        weightInitializer,
        currentState["PRETRAINED_WEIGHTS"],
        optimizer,
        dropout,
        learnRatio,
        regularizer,
        loss,
        debug=currentState["DEBUG"])
    else:
      model = None

  if (not os.path.exists(modelPlotPath)):
    plot_model(model,
           to_file=modelPlotPath,
           show_shapes=True,
           show_layer_names=False)
  history = model.fit(trainIter,
            epochs=currentState["EPOCHS"],
            shuffle=True,
            validation_data=(Xvalidation, Yvalidation),
            callbacks=callbacksList,
            verbose=0)

  PlotHistory(currentState, figPath, history)

  def __innerCalc(results):
    return 0.1 * (1.0 / results[0]) + 0.4 * (results[1] * 100.0) + \
      0.1 * (results[2] * 100.0) + 0.1 * (results[3] * 100.0) + \
      0.1 * (results[4] * 100.0) + 0.1 * (results[5] * 100.0) + \
      0.1 * (results[6] * 100.0)

  results = model.evaluate(Xtrain, Ytrain, verbose=0)
  first = copy.copy(results[:])
  fitnessValueTrain = __innerCalc(results)
  results = model.evaluate(Xvalidation, Yvalidation, verbose=0)
  second = copy.copy(results[:])
  fitnessValueValidation = __innerCalc(results)
  results = model.evaluate(Xtest, Ytest, verbose=0)
  third = copy.copy(results[:])
  fitnessValueTest = __innerCalc(results)
  totalFitnessValue = (fitnessValueTrain + fitnessValueValidation +
             fitnessValueTest) / 3.0
  fitnessValue = np.round(totalFitnessValue, 5)

  record = [0] * len(first)
  for q in range(len(record)):
    record[q] = float((first[q] + second[q] + third[q]) / 3.0)

  for q in range(len(record)):
    if (q in [1, 3, 4, 5]):
      record[q] = round(record[q] * 100.0, 2)
    else:
      if (record[q] >= 100): record[q] = round(record[q], 1)
      elif (record[q] >= 10): record[q] = round(record[q], 2)
      elif (record[q] >= 1): record[q] = round(record[q], 3)
      else: record[q] = round(record[q], 4)

  record.insert(7, fitnessValue)  #WS
  if (fitnessValue >= 100): record[7] = 100
  elif (fitnessValue >= 10): record[7] = round(fitnessValue, 2)
  elif (fitnessValue >= 1): record[7] = round(fitnessValue, 3)
  else: record[7] = round(fitnessValue, 4)

  LogEvent(currentState["LOGGER_FILE"], "=" * 50)
  LogEvent(currentState["LOGGER_FILE"], toLog)
  LogEvent(currentState["LOGGER_FILE"], "Template 1: " + temp1)
  LogEvent(currentState["LOGGER_FILE"], "Template 2: " + temp2)
  LogEvent(currentState["LOGGER_FILE"], "HDF5 1: " + name1)
  LogEvent(currentState["LOGGER_FILE"], "HDF5 2: " + name2)
  LogEvent(currentState["LOGGER_FILE"], "Figure Name: " + figName)
  LogEvent(currentState["LOGGER_FILE"], "Logger Name: " + csvName)
  LogEvent(currentState["LOGGER_FILE"], history.history)
  LogEvent(currentState["LOGGER_FILE"], results)
  LogEvent(currentState["LOGGER_FILE"], fitnessValueTrain)
  LogEvent(currentState["LOGGER_FILE"], fitnessValueValidation)
  LogEvent(currentState["LOGGER_FILE"], fitnessValueTest)
  LogEvent(currentState["LOGGER_FILE"], fitnessValue)
  LogEvent(currentState["LOGGER_FILE"], "=" * 50)

  if (currentState["DEBUG"]):
    print(fitnessValueTrain, fitnessValueValidation, fitnessValueTest,
        fitnessValue)
    print("=" * 50)

  del model, history, aug
  gc.collect()

  return fitnessValue, copy.copy(record)

## PopulationBasedOptimizer Class

In [ ]:
class PopulationBasedOptimizer(object):
  def __init__(self,
         fitnessFunction,
         currentState,
         lowerBound=0.0,
         upperBound=1.0):
    self.fitnessFunction = fitnessFunction
    self.currentState = currentState
    self.lowerBound = self.currentState["LOWER_BOUND"]
    self.upperBound = self.currentState["UPPER_BOUND"]

    self.population = []
    self.populationWithScores = []
    self.currentIteration = 0

  def GetPopulation(self):
    return self.population.copy()

  def GetPopulationWithScores(self):
    return self.populationWithScores

  def InitPopulation(self):
    if (isinstance(self.currentState["DIM"], int)):
      R = self.currentState["DIM"]
    else:
      R = len(self.currentState["DIM"])
    if (not isinstance(self.lowerBound, list)):
      lb = [self.lowerBound for _ in range(R)]
    if (not isinstance(self.upperBound, list)):
      ub = [self.upperBound for _ in range(R)]
    lb = np.asarray(lb)
    ub = np.asarray(ub)

    size = (self.currentState["POPULATION_SIZE"], R)
    uniformRandom = np.random.uniform(0, 1, size)
    clippedUniformRandom = [x * (ub - lb) + lb for x in uniformRandom]
    self.population = np.round(np.asarray(clippedUniformRandom), 3)

  def CalculateFitnessScores(self):
    populationWithFitnessValues = []
    population = self.GetPopulation()
    for j, solution in enumerate(population):
      if (self.currentState["DEBUG"]):
        print("=" * 50)
        print(f"WORKING ON SOLUTION NO. {j+1}.")
        print("=" * 50)
        if (self.currentState["IS_DL"]):
          print(solution, MapSolutionAsString(self.currentState,
                          solution))
      fitnessScore, record = self.fitnessFunction(
        self.currentIteration, solution, self.currentState)
      rec = [solution, fitnessScore, record]
      populationWithFitnessValues.append(rec)
      if (self.currentState["DEBUG"]):
        print("=" * 50)
        print(f"SOLUTION NO. {j+1} HAS SCORE OF {fitnessScore}.")
        print("=" * 50)
    if (self.currentState["IS_MAX_PROBLEM"]): # Maximization.
      populationWithFitnessValues = sorted(populationWithFitnessValues,
                       key=lambda s: s[1],
                       reverse=True)
    else: # Minimization
      populationWithFitnessValues = sorted(populationWithFitnessValues,
                       key=lambda s: s[1],
                       reverse=False)
    self.populationWithScores = populationWithFitnessValues
    bestSolution = populationWithFitnessValues[0]
    if (self.currentState["DEBUG"]):
      print("=" * 50)
      print(f"BEST SOLUTION VALUES = {bestSolution[0]}.")
      print(f"BEST SOLUTION SCORE = {bestSolution[1]}.")
      print("=" * 50)
    LogPaths(self.currentState, self.currentIteration, 
             bestSolution[0], bestSolution[1])

  def Run(self):
    if (self.currentState["SKIP"] == 0
        and self.currentState["POPULATION"] is None):
      if (self.currentState["DEBUG"]): print("Population is created!")
      self.InitPopulation()
    else:
      if (self.currentState["DEBUG"]): print("Population is loaded!")
      self.population = self.currentState["POPULATION"]

    for i in range(self.currentState["SKIP"], self.currentState["ITERS"]):
      if (self.currentState["DEBUG"]):
        print("=" * 50)
        print(f"WORKING ON ITERATION NO. {i+1}.")
        print("=" * 50)
      self.currentIteration = i + 1
      self.CalculateFitnessScores()
      self.LogHistory()
      LogEvent(self.currentState["LOGGER_FILE"],
           f"Old Population @ {i+1}")
      LogEvent(self.currentState["LOGGER_FILE"], self.GetPopulation())
      self.UpdatePopulation()
      LogEvent(self.currentState["LOGGER_FILE"],
           f"New Population @ {i+1}")
      LogEvent(self.currentState["LOGGER_FILE"], self.GetPopulation())
      self.UpdateConfigurations()

    LogEvent(self.currentState["LOGGER_FILE"], "=" * 50)
    LogEvent(self.currentState["LOGGER_FILE"], "Completed")
    LogEvent(self.currentState["LOGGER_FILE"], "=" * 50)

    if (currentState["DEBUG"]): print("Completed!")

  def PrintPopulation(self):
    if (self.currentState["DEBUG"] and self.currentState["IS_DL"]):
      for j, temp in enumerate(self.GetPopulation()):
        mappedSolution = MapSolutionForCNN(self.currentState, temp)
        print(self.currentIteration, j + 1, temp, mappedSolution)

  def LogHistory(self):
    LogEvent(self.currentState["LOGGER_FILE"], "=" * 50)
    LogEvent(self.currentState["LOGGER_FILE"], self.currentIteration)
    LogEvent(self.currentState["LOGGER_FILE"], self.GetPopulation())
    LogEvent(self.currentState["LOGGER_FILE"],
         self.GetPopulationWithScores())
    if (self.currentState["IS_DL"]):
      for j, solution in enumerate(self.GetPopulationWithScores()):
        mappedSolution = MapSolutionForCNN(self.currentState, solution[0])
        LogEvent(self.currentState["LOGGER_FILE"],
             [j + 1, mappedSolution, solution[1]])
      LogEvent(self.currentState["LOGGER_FILE"], "=" * 50)

    bestSolution = self.GetPopulationWithScores()[0]
    mappedBestSolution = MapSolutionAsString(self.currentState,
                         bestSolution[0])
    mappedBestSolution.extend(bestSolution[2])
    if (self.currentState["IS_DL"]):
      LogBestSolution(currentState, self.currentIteration,
            mappedBestSolution)

  def UpdateConfigurations(self):
    self.currentState["SKIP"] = self.currentIteration
    self.currentState["POPULATION"] = self.GetPopulation().copy()
    self.StoreWorkStateNow()

  def StoreWorkStateNow(self):
    StoreWorkState(self.currentState, self.currentState["STATE_PICKLE"])

  @abstractmethod
  def UpdatePopulation(self):
    raise NotImplementedError(
      "UpdatePopulation function must be implemented.")

## MantaRayForagingOptimizer Class

In [ ]:
class MantaRayForagingOptimizer(PopulationBasedOptimizer):
  def UpdatePopulation(self):
    if (isinstance(self.currentState["DIM"], int)):
      dim = self.currentState["DIM"]
    else:
      dim = len(self.currentState["DIM"])
    iters = self.currentState["ITERS"]
    populationSize = self.currentState["POPULATION_SIZE"]
    lb = self.lowerBound
    ub = self.upperBound

    scores = self.GetPopulationWithScores()
    bestSolution, bestFitness = scores[0][0], scores[0][1]
    for j, temp in enumerate(scores):
      fitness = scores[j][1]
      if (self.currentState["IS_MAX_PROBLEM"]): # Maximization.
        if (fitness > bestFitness):
          bestSolution, bestFitness = scores[j][0], scores[j][1]
      else:
        if (fitness < bestFitness):
          bestSolution, bestFitness = scores[j][0], scores[j][1]

    if not isinstance(lb, list):
      lb = [lb for _ in range(dim)]
      ub = [ub for _ in range(dim)]
    lb = np.asarray(lb)
    ub = np.asarray(ub)

    X = np.asarray([e[0].copy() for e in scores])

    newX = X.copy()

    coef = self.currentIteration / iters

    for i in range(0, populationSize):
      r = np.random.random() 
      alpha = 2.0 * r * np.sqrt(np.abs(np.log(r)))

      r1 = np.random.random() 
      factor = (iters - self.currentIteration + 1.0) / (iters * 1.0)
      beta = 2.0 * np.exp(r1 * factor) * np.sin(2.0 * np.pi * r1)

      if (np.random.random() < 0.5):
        if (coef < np.random.random()):
          xRand = lb + np.random.random(dim) * (ub - lb)
          if (i == 0):
            newX[i, :] = xRand + r * (xRand - X[i, :]) + beta * (
              xRand - X[i, :])
          else:
            newX[i, :] = xRand + r * (
              X[i - 1, :] - X[i, :]) + beta * (xRand - X[i, :])
        else:
          if (i == 0):
            newX[i, :] = bestSolution + r * (bestSolution - X[
              i, :]) + beta * (bestSolution - X[i, :])
          else:
            newX[i, :] = bestSolution + r * (X[i - 1, :] - X[
              i, :]) + beta * (bestSolution - X[i, :])
      else:
        if (i == 0):
          newX[i, :] = X[i, :] + r * (bestSolution - X[
            i, :]) + alpha * (bestSolution - X[i, :])
        else:
          newX[i, :] = X[i, :] + r * (X[i - 1, :] - X[
            i, :]) + alpha * (bestSolution - X[i, :])

      newX[i, :] = np.clip(newX[i, :], lb, ub)
      currentScore = self.fitnessFunction(self.currentIteration,
                        newX[i, :],
                        self.currentState)[0]

      if (self.currentState["IS_MAX_PROBLEM"]): # Maximization.
        if (currentScore > bestFitness):
          bestSolution, bestFitness = newX[i, :], currentScore
      else:
        if (currentScore < bestFitness):
          bestSolution, bestFitness = newX[i, :], currentScore

      s = 2.0
      r2 = np.random.random()
      r3 = np.random.random() 
      newX[i, :] = X[i, :] + s * (r2 * bestSolution - r3 * X[i, :])

      newX[i, :] = np.clip(newX[i, :], lb, ub)
      currentScore = self.fitnessFunction(self.currentIteration,
                        newX[i, :],
                        self.currentState)[0]

      if (self.currentState["IS_MAX_PROBLEM"]): # Maximization.
        if (currentScore > bestFitness):
          bestSolution, bestFitness = newX[i, :], currentScore
      else:
        if (currentScore < bestFitness):
          bestSolution, bestFitness = newX[i, :], currentScore

    X = newX.copy()
    random.shuffle(X)
    X = np.clip(X, lb, ub)
    self.population = np.round(np.asarray(X), 3)

# Learning and Optimization Configurations

In [ ]:
OWNER               = "..."     
PBO                 = "MRFO"
IS_DL               = True
IS_MAX_PROBLEM      = True
DATASET_PICKLE_FILE = "PUT_YOUR_DATASET_PICKLE_FILE_NAME_HERE.p"
PICKELS_BASE_DIR    = "..." 
RESULTS_BASE_DIR    = "..." 
IMAGE_SHAPE         = (64, 64, 3)        
CATEGORIES          = ("NORMAL", "PNEUMONIA-Viral", "PNEUMONIA-Bacterial", "COVID-19")
ITERS               = 1000    
EPOCHS              = 8      
POPULATION_SIZE     = 20
SKIP                = 0       
POPULATION          = None    
IS_NEW              = False   
FF                  = FitnessFunction
LOWER_BOUND         = 0.0
UPPER_BOUND         = 1.0
DIM                 = 4
USE_TF              = True
MODEL               = "MobileNet" 
TRAIN_RATIO         = 0.85    
DROPOUT_RANGE       = [0, 6]
OUTPUT_ACTIVATION   = "softmax"
LOSS                = "categorical_crossentropy"
PRETRAINED_WEIGHTS  = "imagenet"
BATCH_SIZES         = list(np.arange(8, 100, 8))
LEARN_RATIOS        = list(np.arange(0, 105, 5))
REGULARIZERS        = []
AUGMENTATION_CONFIG = {
  "rotation_range"     : 15,
  "width_shift_range"  : 0.15,
  "height_shift_range" : 0.15,
  "shear_range"        : 0.15,
  "zoom_range"         : 0.15,
  "horizontal_flip"    : True,
  "vertical_flip"      : False,
  "brightness_range"   : None,
  "fill_mode"          : 'nearest'
}
OPTIMIZERS          = [
  "adam", "nadam", "adagrad", "adadelta", 
  "adamax", "rmsprop", "sgd", "ftrl",
]
HIDDEN_ACTIVATIONS  = []
WEIGHT_INITIALIZERS = []
METRICS_LIST        = { 
  "accuracy"                                                              : True,         
  f1_m                                                                    : True,                             
  precision_m                                                             : True,                      
  recall_m                                                                : True,                         
  specificity_m                                                           : True,                    
  AUC(name="hmb_auc")                                                     : True,                            
  sensitivity_m                                                           : True,                    
  iou_coef_m                                                              : True,                       
  dice_coef_m                                                             : True,                      
  Precision(name="hmb_precision")                                         : True,                      
  Recall(name="hmb_recall")                                               : True,                         
  TruePositives(name="hmb_true_positive")                                 : True,         
  TrueNegatives(name="hmb_true_negative")                                 : True,         
  FalsePositives(name="hmb_false_positive")                               : True,        
  FalseNegatives(name="hmb_false_negative")                               : True,        
}

In [ ]:
RESULTS_DIR = os.path.join(BASE_DIR, RESULTS_BASE_DIR)
DATASET_PICKLE = os.path.join(BASE_DIR, PICKELS_BASE_DIR, DATASET_PICKLE_FILE)

OWNER_DIR = os.path.join(RESULTS_DIR, OWNER)
if (not os.path.exists(OWNER_DIR)): os.mkdir(OWNER_DIR)

HDF5_RESULTS_DIR = os.path.join(OWNER_DIR, f"Models-{OWNER}")
FIGURES_RESULTS_DIR = os.path.join(OWNER_DIR, f"Figures-{OWNER}")
HISTORY_RESULTS_DIR = os.path.join(OWNER_DIR, f"History-{OWNER}")
STATE_PICKLE = os.path.join(OWNER_DIR, f"State-{OWNER}.p")
LOGGER_FILE = os.path.join(OWNER_DIR, f"Logger-{OWNER}.txt")
PATHS_FILE = os.path.join(OWNER_DIR, f"Paths-{OWNER}.csv")
BEST_FILE = os.path.join(RESULTS_DIR, f"Best-{OWNER}.csv")

SPLIT_DATASET_PICKLE = os.path.join(BASE_DIR, PICKELS_BASE_DIR,
                  f"Split-{DATASET_PICKLE_FILE}")

if (not os.path.exists(HDF5_RESULTS_DIR)): os.mkdir(HDF5_RESULTS_DIR)
if (not os.path.exists(FIGURES_RESULTS_DIR)): os.mkdir(FIGURES_RESULTS_DIR)
if (not os.path.exists(HISTORY_RESULTS_DIR)): os.mkdir(HISTORY_RESULTS_DIR)

In [ ]:
METRICS_KEYWORDS = []
METRICS_NAMES = []
METRICS = [d[0] for d in METRICS_LIST.items() if d[1]]
if (IS_DL): 
  for m in METRICS:
    if ("tensorflow.python.keras.metrics" in str(m)):
      METRICS_NAMES.append(m.name)
      nameList = m.name.strip().split("_")[1:]
      nameList = [s.capitalize() for s in nameList]
      name = " ".join(nameList)
    elif (hasattr(m, '__call__')):
      name = str(m).split()[1].strip()
      METRICS_NAMES.append(name)
      nameList = name.strip().split("_")
      nameList = [s.capitalize() for s in nameList]
      name = " ".join(nameList)
    elif (isinstance(m, str)):
      METRICS_NAMES.append(m)
      name = m.strip().capitalize()
    METRICS_KEYWORDS.append(name)
  assert (len(METRICS) >= 6)
  assert (len(METRICS) == len(METRICS_KEYWORDS) == len(METRICS_NAMES))

In [ ]:
currentState = HandleCurrentState(IS_NEW, STATE_PICKLE, LOGGER_FILE, DEBUG)

if (IS_DL):
  Q = LoadCreateSplitFile(currentState["DATASET_PICKLE"],
              currentState["SPLIT_DATASET_PICKLE"],
              len(currentState["CATEGORIES"]),
              currentState["TRAIN_RATIO"], currentState["DEBUG"])
  [Xtrain, Xvalidation, Xtest, Ytrain, Yvalidation, Ytest] = Q

  if (currentState["DEBUG"]):
    print(Xtrain.shape, Ytrain.shape)
    print(Xtest.shape, Ytest.shape)
    print(Xvalidation.shape, Yvalidation.shape)

  LogEvent(currentState["LOGGER_FILE"], "=" * 50)
  LogEvent(currentState["LOGGER_FILE"], Xtrain.shape)
  LogEvent(currentState["LOGGER_FILE"], Ytrain.shape)
  LogEvent(currentState["LOGGER_FILE"], Xvalidation.shape)
  LogEvent(currentState["LOGGER_FILE"], Yvalidation.shape)
  LogEvent(currentState["LOGGER_FILE"], Xtest.shape)
  LogEvent(currentState["LOGGER_FILE"], Ytest.shape)
  LogEvent(currentState["LOGGER_FILE"], "=" * 50)

  PlotRandomFigure(Xtrain, Ytrain, currentState["CATEGORIES"],
           currentState["DEBUG"])

# PBO Checking and Object Creation 

In [ ]:
if (currentState["PBO"] == "MRFO"):
  pbo = MantaRayForagingOptimizer(FF, currentState)
else:
  print("Incorrect PBO Value.")

# Run

In [ ]:
if (pbo): pbo.Run()
else: print("Are you kidding?")